<a href="https://colab.research.google.com/github/guptapak/NewHacks/blob/main/FloodScan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import the Python SDK
!pip install reportlab
import google.generativeai as genai
from IPython.display import Markdown
!pip install reportlab
import json

def resp_to_dict(response):
  return json.loads(response.text.replace("```python", "").replace("```","").replace("\n","").replace(" ","").replace("\\",""))

# Used to securely store your API key
from google.colab import userdata


GOOGLE_API_KEY=userdata.get('KEY')
genai.configure(api_key=GOOGLE_API_KEY)


model = genai.GenerativeModel('gemini-pro')


# Upload the file and print a confirmation.
sample_file = genai.upload_file(path="Before_flood.png",
                                display_name="Reference")


print(f"Uploaded file '{sample_file.display_name}' as: {sample_file.uri}")

file = genai.get_file(name=sample_file.name)
print(f"Retrieved file '{file.display_name}' as: {sample_file.uri}")

# Choose a Gemini API model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Prompt the model with text and the previously uploaded image.
#response = model.generate_content([sample_file, "insert prompt here"])

response_1 = model.generate_content([sample_file, """Analyze the image I provide and identify the objects present in it. Output should be in json format. For each object, give an output in the form of a Python dictionary:

python
Copy code
{
    "object_name": {
        "count": <number of objects detected>,
        "broken": <number of broken objects>,
        "normal": <number of normal objects>
    }
}
Use the object's name as the key, and provide values for "count," "broken," and "normal" based on what you observe in the image. Even If an attribute does not apply to an object, do not omit it from the dictionary."""])

resp_1 = resp_to_dict(response_1)

r1s = str(resp_1)

# Upload the file and print a confirmation.
sample_file_2 = genai.upload_file(path="After_flood.png",
                                display_name="Test")

print(f"Uploaded file '{sample_file_2.display_name}' as: {sample_file_2.uri}")

file = genai.get_file(name=sample_file_2.name)
print(f"Retrieved file '{file.display_name}' as: {sample_file_2.uri}")

# Prompt the model with text and the previously uploaded image.
#response = model.generate_content([sample_file, "insert prompt here"])

s = """If an object is same as the objects in the image beofre this, use the same name, do not use different names for the same object that appears in both the images.Analyze the image I provide and identify the objects present in it. Output should be in json format For each object, give an output in the form of a Python dictionary:

python
Copy code
{
    "object_name": {
        "count": <number of objects detected>,
        "broken": <number of broken objects>,
        "normal": <number of normal objects>
    }
}
Use the object's name as the key, and provide values for "count," "broken," and "normal" based on what you observe in the image. Even If an attribute does not apply to an object, do not omit it from the dictionary.""" + r1s

response_2 = model.generate_content([sample_file_2, s])



# #print(a)

# b = []

# for i in response:
#     b.append(i)
# print(b)
# Display response_1 and response_2 outputs
display(Markdown("### Response for Before Flood Image"))
display(Markdown(">" + response_1.text))

display(Markdown("### Response for After Flood Image"))
display(Markdown(">" + response_2.text))

# def resp_to_dict(response):
#   return json.loads(response.text.replace("```python", "").replace("```","").replace("\n","").replace(" ","").replace("\\",""))
resp_2 = resp_to_dict(response_2)
missing_keys = set(resp_1.keys()) - set(resp_2.keys())


# Function to compare before and after flood dictionaries
def find_missing_objects(resp_1, resp_2):
    missing_objects = {}

    for obj, details in resp_1.items():
        if obj in resp_2:
            # Compare the counts in both dictionaries
            missing_count = details['count'] - resp_2[obj]['count']
            if missing_count > 0:
                missing_objects[obj] = missing_count
        else:
            # If the object doesn't exist in resp_2, it means all are missing
            missing_objects[obj] = details['count']

    return missing_objects

# Call the function to find missing objects
missing_objects = find_missing_objects(resp_1, resp_2)

# Print the missing objects and the number of missing items
print("Missing objects and counts:", missing_objects)

# Find broken objects
def find_broken_objects(resp_2):
    broken_objects = {}

    # Iterate through the objects in the after-flood response
    for obj, details in resp_2.items():
        # Check if the number of broken items is greater than 0
        if details['broken'] > 0:
            broken_objects[obj] = details['broken']

    return broken_objects
# Call the function to find broken objects
broken_objects = find_broken_objects(resp_2)

# Print the broken objects and the number of broken items
print("Broken objects and counts:", broken_objects)

!pip install reportlab
import os
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from google.colab import files

# Create the directory if it doesn't exist
os.makedirs('/mnt/data', exist_ok=True)

# Define the file path for the PDF
file_path = '/mnt/data/insurance_claim.pdf'
pdf = SimpleDocTemplate(file_path, pagesize=letter)

# Table header
data = [
    ["Item", "Qty", "Missing","Broken","Replacement Cost", "Total Cost"]
]

# Example replacement costs, you may adjust this as needed
replacement_costs = {
    "chair": 50.0,
    "table": 150.0,
    "floorlamp": 100.0,
    "pendantlight": 75.0
}
# Combine missing and broken items into a single dictionary
combined_objects = {}
for item, qty in missing_objects.items():
    combined_objects[item] = {
        'missing': qty,
        'broken': 0  # Set broken to 0 initially
    }

for item, qty in broken_objects.items():
    if item in combined_objects:
        combined_objects[item]['broken'] = qty
    else:
        combined_objects[item] = {
            'missing': 0,  # Set missing to 0 if it's not missing
            'broken': qty
        }

# Create the directory if it doesn't exist
os.makedirs('/mnt/data', exist_ok=True)

# Define the file path for the PDF
file_path = '/mnt/data/insurance_claim.pdf'
pdf = SimpleDocTemplate(file_path, pagesize=letter)

# Table header
data = [
    ["Item","Total Qty", "Missing Qty", "Broken Qty", "Replacement Cost", "Total Cost"]
]

# Add each missing/broken object to the table
for item, counts in combined_objects.items():
    missing_qty = counts['missing']
    broken_qty = counts['broken']
    total_qty = missing_qty + broken_qty
    replacement_cost_per_item = replacement_costs.get(item, 0)
    total_cost = total_qty * replacement_cost_per_item
    data.append([item, str(total_qty),str(missing_qty), str(broken_qty), str(replacement_cost_per_item), str(total_cost)])


# Add each missing object to the table
#for item, qty in missing_objects.items() or broken_objects.items():
#    total_cost = qty * replacement_cost_per_item
#    data.append([item, str(qty), str(replacement_cost_per_item), str(total_cost)])

# Create a table with data
table = Table(data)

# Style the table
style = TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, -1), 10),
    ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
    ('BACKGROUND', (0, 1), (-1, -1), colors.white),
    ('GRID', (0, 0), (-1, -1), 1, colors.black),
])

table.setStyle(style)

# Build the PDF
elements = [table]
pdf.build(elements)

# Download the PDF
files.download(file_path)



Uploaded file 'Reference' as: https://generativelanguage.googleapis.com/v1beta/files/iegjnj9ohy82
Retrieved file 'Reference' as: https://generativelanguage.googleapis.com/v1beta/files/iegjnj9ohy82
Uploaded file 'Test' as: https://generativelanguage.googleapis.com/v1beta/files/n1pf8jj3nkjj
Retrieved file 'Test' as: https://generativelanguage.googleapis.com/v1beta/files/n1pf8jj3nkjj


### Response for Before Flood Image

>```python
{
  "floor lamp": {
    "count": 1,
    "broken": 0,
    "normal": 1
  },
  "chair": {
    "count": 2,
    "broken": 0,
    "normal": 2
  },
  "table": {
    "count": 1,
    "broken": 0,
    "normal": 1
  },
  "pendant light": {
    "count": 1,
    "broken": 0,
    "normal": 1
  }
}
```

### Response for After Flood Image

>```python
{
  "floor lamp": {
    "count": 1,
    "broken": 0,
    "normal": 1
  },
  "chair": {
    "count": 1,
    "broken": 1,
    "normal": 0
  },
  "pendant lamp": {
    "count": 1,
    "broken": 0,
    "normal": 1
  }
}
```


Missing objects and counts: {'chair': 1, 'table': 1, 'pendantlight': 1}
Broken objects and counts: {'chair': 1}


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>